In [6]:
# Lets start with agent_toolkits

data_json = "task_analyse3.json"

pip install langchain langchain-openai beautifulsoup4 wikipedia langchainhub langserve[all] langchain-community python-dotenv tavily-python

In [2]:
import json
from langchain.agents import create_json_agent
from langchain_community.agent_toolkits import JsonToolkit
from langchain_community.tools.json.tool import JsonSpec
from langchain_openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv("D:\\gitFolders\\python_de_learners_data\\.env")

True

In [8]:
with open(data_json, 'r') as jds:
    data = json.load(jds)

js_spec = JsonSpec(dict_=data, max_value_length=4000)
js_tkit = JsonToolkit(spec=js_spec)

In [9]:
print(js_tkit.spec.dict_)

{'app_type': 'command line app', 'name': 'text_processor', 'prompt': 'My app will open a text file, read it and help to search for specific words, and sentences inside the text. It will act similar to the grep command in linux. The user can provide the file name to process or he can open the file content and pipe it to this app. ', 'classified_tasks': [{'task': 'The "text_processor" command line app will open a text file, read its content, and allow users to search for specific words and sentences within the text. Users can provide the file name as an argument to process or pipe the file content directly to the app.', 'task_class': 'code executing'}, {'task': '1. As a user, I want to be able to run the "text_processor" app from the command line.', 'task_class': 'code executing'}, {'task': '2. As a user, I want to be able to specify the file name as an argument to process a specific text file.', 'task_class': 'clarification request'}, {'task': '3. As a user, I want to be able to pipe th

In [10]:
json_agent = create_json_agent(
    llm=OpenAI(api_key=os.environ['OPENAI_API_KEY']),
    toolkit=js_tkit,
    verbose=True
)

In [11]:
json_agent.run("How many tasks are present in the data")

d:\tempVids\onlylc\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['app_type', 'name', 'prompt', 'classified_tasks', 'task_analysis']
Thought: I should look at the value associated with the key 'classified_tasks'
Action: json_spec_get_value
Action Input: data['classified_tasks']
Observation: [{'task': 'The "text_processor" command line app will open a text file, read its content, and allow users to search for specific words and sentences within the text. Users can provide the file name as an argument to process or pipe the file content directly to the app.', 'task_class': 'code executing'}, {'task': '1. As a user, I want to be able to run the "text_processor" app from the command line.', 'task_class': 'code executing'}, {'task': '2. As a user, I want to be able to specify the file name as an argument to process a specific text file.', 'task_class': 'clarification request'}, {'task': '3. As a user, I want to be able to pipe the content of a text file di

'19'

#### Diving into Agents

AgentAction: Represents Action to be taken, with a tool and tool_input property

AgentFinish: Represents final results from an agent, contain return_values (dict) which contains 'output' key

Intermediate Steps: Represents 'Prev actions' & corresponding outputs from this Current Agent Run.

Agent : Is a Chain 

LLM --> Prompt --> Output Parser

AgentInput: KV mapping which requires 'Intermediate_steps' key

AgentOutput: Next Action to take or Final Outputs to send to User (AgentAction, 
AgentFinish)

Main thing affects Agent is the prompting strategy used

The goal of the OpenAI tools APIs is to more reliably return valid and useful function calls than what can be done using a generic text completion or chat API.

In [12]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_community.tools.tavily_search import TavilySearchResults

In [13]:
tools = [TavilySearchResults(max_results=1)]

In [21]:
print(tools[0].name)
print(tools[0].get_prompts())


tavily_search_results_json
[]


In [14]:
tools[0].run(tool_input="Where is moon",
             color='yellow',
             run_name='tester')

[{'url': 'https://theskylive.com/moon-info',
  'content': 'Closest Approach of The Moon to Earth\nBetween 1 January 1600 and 30 December 2499,\nthe closest approach of\nThe Moon to Earth happens on\nWed Jan 13 2275 at a distance of\n0.002382 Astronomical Units, or\n356,377 kilometers:\nNOTE: values for the closest approach are computed\nwith a sampling interval of 1 day.\n The Moon Physical Data\nThe following shows the known values of the most important physical parameters of The Moon.\nSource: JPL Small-Body Database\nThe Moon 15 Days Ephemeris\n Support us\nThe Moon Ephemeris Calculator\nCompute the position of The Moon\t\t\t\t\tfor any date and time between 1 January 1600\nand 30 December 2499\nand display the results on an interactive star map.\n The Moon Position and Finder Charts\nBelow we provide\nThe Moon finder charts showing where the object is right now in the sky with respect to the brightest\nstars. The Moon Ephemeris Calculator\nCompute the position of The Moon\t\t\t\tfo

In [15]:
# Get the prompt to use - you can modify this!
# https://smith.langchain.com/hub/hwchase17?organizationId=f590798d-fdf9-4f3f-aeff-1ec23aea38a0
jsonprompt = hub.pull("hwchase17/react-chat-json")
print(jsonprompt.input_variables)
print(jsonprompt.metadata)
print(jsonprompt.messages[0].prompt.template)

['agent_scratchpad', 'input', 'tool_names', 'tools']
{'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react-chat-json', 'lc_hub_commit_hash': '9c1258e8aa8ce33bebbd62e077c143d0b06c81f3c7de732187ee61c70c1254c7'}
Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on th

In [16]:
from langchain.agents import create_json_chat_agent

json_agent = create_json_chat_agent(OpenAI(api_key=os.environ['OPENAI_API_KEY']),
                                    tools, 
                                    jsonprompt)

{'tools': 'tavily_search_results_json: A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'tool_names': 'tavily_search_results_json'}
[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to pro

In [17]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=json_agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [18]:
agent_executor.invoke({"input": "when was the sun born?"})



> Entering new AgentExecutor chain...


```json
{
    "action": "tavily_search_results_json",
    "action_input": "When was the sun born?"
}
```[{'url': 'https://en.wikipedia.org/wiki/Sun', 'content': 'Theoretical models of the Sun\'s interior indicate a maximum power density, or energy production, of approximately 276.5 watts per cubic metre at the center of the core,[65] which, according to Karl Kruszelnicki, is about the same power density inside a compost pile.[66]\nThe fusion rate in the core is in a self-correcting equilibrium: a slightly higher rate of fusion would cause the core to heat up more and expand slightly against the weight of the outer layers, reducing the density and hence the fusion rate and correcting the perturbation; and a slightly lower rate would cause the core to cool and shrink slightly, increasing the density and increasing the fusion rate and again reverting it to its present rate.[67][68]\nRadiative zone\nThe radiative zone is the thickest layer of the s

{'input': 'when was the sun born?',
 'output': 'The response to your last comment is that the Sun is primarily composed of hydrogen and helium, and its core is where the majority of its energy is produced through nuclear fusion. It also has a complex structure and interacts with the surrounding interstellar medium and other celestial bodies in our solar system.'}